In [1]:
!pip install torch datasets transformers huggingface_hub wandb hf_transfer

In [2]:
import os
import sys
import torch
import wandb
from torch.amp import autocast, GradScaler
from torch.optim import AdamW
import math, time
from create_shards import split_and_shard, load_split
from model import BasicGPT, ModelConfig
from create_dataloaders import build_packed_dataloaders
from transformers import AutoTokenizer

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
from huggingface_hub import login
sys.path.append("/workspace/.local")
login()

In [4]:
!hf auth whoami

user:  timdadum


In [5]:
os.environ["WANDB_API_KEY"] = "key"
wandb.init(project="runpod-llm-recursion")
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="timdadum-personal",
    # Set the wandb project where this run will be logged.
    project="my-awesome-project",
    # Track hyperparameters and run metadata.
    # config={
    #     "learning_rate": 0.02,
    #     "architecture": "CNN",
    #     "dataset": "CIFAR-100",
    #     "epochs": 10,
    # },
)

wandb: Currently logged in as: timdadum (timdadum-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
def _extract_loss(output):
    """
    Robustly extract a scalar loss from various output shapes:
    - CausalLMOutput-like objects (have .loss)
    - dicts with 'loss'
    - tuples/lists with loss in first position
    """
    if hasattr(output, "loss") and output.loss is not None:
        return output.loss
    if isinstance(output, dict) and "loss" in output:
        return output["loss"]
    if isinstance(output, (tuple, list)) and len(output) > 0:
        return output[0]
    raise ValueError("Could not extract loss from model output.")

@torch.no_grad()
def evaluate(dataloader, max_batches=None):
    model.eval()
    losses, n = 0.0, 0

    for i, batch in enumerate(dataloader):
        if max_batches and i >= max_batches:
            break

        ids = batch.get("input_ids").to(device)
        mask = batch.get("attention_mask")
        labels = batch.get("labels")

        if mask is not None:
            mask = mask.to(device)
        if labels is not None:
            labels = labels.to(device)

        out = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = _extract_loss(out)
        losses += float(loss.item())
        n += 1

    model.train()
    avg = losses / max(1, n)
    return avg, math.exp(avg)

def train(config, train_loader, eval_loader, run_name, epochs=3, log_every=50, eval_every=500):
    global_step = 0
    best_perplexity = float("inf")
    print("[TRAINING]: Starting training...")
    model.to(device).train()
    t0 = time.time()
    running = 0.0

    for epoch in range(epochs):
        print(f"[TRAINING]: Starting epoch {epoch+1}/{epochs}...")
        for step, batch in enumerate(train_loader, start=1):
            # Prepare tensors (guard for None masks)
            x = batch.get("input_ids").to(device)
            mask = batch.get("attention_mask")
            if mask is not None:
                mask = mask.to(device)

            # Forward + loss
            optimizer.zero_grad(set_to_none=True)
            amp_ctx = autocast(device_type=device, enabled=(device == "cuda" and config.amp))
            with amp_ctx:
                out = model(input_ids=x, attention_mask=mask, labels=x)
                loss = _extract_loss(out)

            wandb.log({"train_loss": loss, "epoch": epoch})

            # Backward + update
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
            scaler.step(optimizer)
            scaler.update()

            global_step += 1
            running += float(loss.item())  # accumulate for logging window

            # Log every N steps (use the function arg 'log_every', not config.log_every)
            if global_step % log_every == 0:
                print(f"[TRAINING]: epoch {epoch+1} | step {global_step} | loss {running / log_every:.4f}")
                running = 0.0

            # Eval every M steps
            if global_step % eval_every == 0:
                eval_loss, eval_ppl = evaluate(eval_loader)
                print(f"[TRAINING - EVAL] step {global_step} | val_loss {eval_loss:.4f} | val_ppl {eval_ppl:.2f}")
                if eval_ppl < best_perplexity:
                    best_perplexity = eval_ppl
                    path = os.path.join(config.out_dir, f"{run_name}_best.pt")
                    torch.save({
                        "config": vars(config),
                        "model_state": model.state_dict(),
                        "optimizer_state": optimizer.state_dict(),
                        "step": global_step,
                        "val_ppl": eval_ppl,
                    }, path)
                    print("Saved best:", path)

    # Final eval + save
    final_eval_loss, final_eval_ppl = evaluate(eval_loader)
    elapsed = time.time() - t0
    print(f"Epoch {epochs} in {elapsed:.1f}s | val_loss {final_eval_loss:.4f} | val_ppl {final_eval_ppl:.2f}")
    path = os.path.join(config.out_dir, f"{run_name}_e{epochs}_step{global_step}.pt")
    torch.save({
        "config": vars(config),
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "step": global_step,
        "val_ppl": final_eval_ppl,
    }, path)
    print("Saved:", path)

In [ ]:
DO_TRAIN = False
BASE = f"/recursion"
config = ModelConfig(
    d_model = 1028,
    max_len = 512,
    project_name = "basic-llm-d512-l512-10e-[D]25%en"
)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if input("Train? (Y/N)").lower() == 'y':
  DO_TRAIN = True

  if input("Load existing model? (Y/N)") == 'y':
    model, checkpoint = BasicGPT.load_weights(f"{BASE}/trained_models/simple_baseline_best.pt", config)
    model.to(device)
    print("Successfully loaded model weights from earlier run. Continuing...")
  else:
    print("Initialized model. Continuing...")    
    model = BasicGPT(config)
    print(model)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model params: {total_params/1e6:.2f}M")

  split_and_shard(
    ds_name="manu/project_gutenberg", # "Cohere/wikipedia-2023-11-embed-multilingual-v3",
    # config="en",
    hf_split="en",
    out_dir=f"{BASE}/data/shards",
    max_shard_size="1024MB",
    num_proc=16
  )

  train_loader = load_split(f"{BASE}/data/shards", "train", shard_by_rank=True, shuffle_seed=42)
  eval_loader   = load_split(f"{BASE}/data/shards", "val",   shard_by_rank=True, shuffle_seed=43)
  test_loader  = load_split(f"{BASE}/data/shards", "test",  shard_by_rank=True, shuffle_seed=44)

  train_loader, eval_loader, test_loader = build_packed_dataloaders(
    train_loader, eval_loader, test_loader,
    tokenizer_name="gpt2",
    seq_len=256,
    batch_size=32,
  )

  # Peek
  xb, yb = next(iter(train_loader))["input_ids"], next(iter(train_loader))["labels"]
  print("Sample batch:", xb.shape, yb.shape)

  print("Sample item (token integers):", xb[1], yb.shape[1])

  text = tokenizer.decode(xb[1].tolist(), skip_special_tokens=False)
  print(f"Decoded sample batch: {text}")

  optimizer = AdamW(model.parameters(),
                    lr=config.lr,
                    betas=config.betas,
                    weight_decay=config.weight_decay)

  scaler = GradScaler(enabled=(device == "cuda" and getattr(config, "amp", True)))

  train(config, train_loader, eval_loader, run_name="simple_baseline", epochs=10)
else:
  if input("Load existing model? (Y/N)") == 'y':
    model, checkpoint = BasicGPT.load_weights(f"{BASE}/trained_models/simple_baseline_best.pt", config)
    print(model)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model params: {total_params/1e6:.2f}M")
    model.to(device)
    print("Successfully loaded model.")


Using device: cuda


Train? (Y/N) y
Load existing model? (Y/N) n


Initialized model. Continuing...
BasicGPT(
  (token_emb): Embedding(50257, 1028)
  (pos_emb): Embedding(512, 1028)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1028, out_features=1028, bias=True)
        )
        (linear1): Linear(in_features=1028, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=1028, bias=True)
        (norm1): LayerNorm((1028,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1028,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm_f): LayerNorm((1028,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=1028, out_features=50257, bias=False

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

data/en-00002-of-00052-835bd07d97f52cbd.(…):   0%|          | 0.00/215M [00:00<?, ?B/s]

...

data/ru-00000-of-00001-c0a99d21d8849748.(…):   0%|          | 0.00/343k [00:00<?, ?B/s]

data/sv-00000-of-00001-6d07271c1c00324d.(…):   0%|          | 0.00/40.5M [00:00<?, ?B/s]

data/zh-00000-of-00001-bca42330c6f1826c.(…):   0%|          | 0.00/120M [00:00<?, ?B/s]

Generating de split:   0%|          | 0/3131 [00:00<?, ? examples/s]

Generating en split:   0%|          | 0/61340 [00:00<?, ? examples/s]

In [33]:
#TODO: Evaluate on test set

def finish_prompt(prompt: str, model: BasicGPT, max_new_tokens: int = 256):
    model.eval()
    device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs.get("attention_mask", None),
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.9,
            eos_token_id=getattr(tokenizer, "eos_token_id", None),
        )

    return tokenizer.decode(output_ids[0].tolist(), skip_special_tokens=True)

In [34]:
prompt = "There once was a man who "
print(finish_prompt(prompt, model, 128))

There once was a man who   tried
                                                                                     CHLEPHOES

                                   
